In [259]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

## Cost matrix

In [260]:
cost_matrix = pd.read_csv('cost_matrix.csv')

# Convert the entire DataFrame to integers
cost_matrix = cost_matrix.astype(int)

# If the index of the DataFrame needs to be integers (e.g., if the index is non-numeric):
cost_matrix.index = cost_matrix.index.astype(int)

# If the column names need to be integers (if they are non-numeric or string-based):
cost_matrix.columns = cost_matrix.columns.astype(int)

In [261]:
cost_matrix

,0,1,2,3,4
0,0,1,3,2,2
1,1,0,2,4,1
2,3,2,0,1,4
3,2,4,1,0,3
4,2,1,4,3,0


In [262]:
# Number of shops to visit
n = cost_matrix.shape[0]

# Calculate M
M = 0
for i in range(n):
    for j in range(n):
        M += cost_matrix.iloc[i,j]

# M cost for the path i->i
for i in range(n):
    cost_matrix.iloc[i,i] = M + 1

In [263]:
cost_matrix

,0,1,2,3,4
0,47,1,3,2,2
1,1,47,2,4,1
2,3,2,47,1,4
3,2,4,1,47,3
4,2,1,4,3,47


## Shop demands

In [264]:
shop_demands = pd.read_csv('shop_demands.csv').to_numpy(dtype=int)

## Closest neighbour heuristic

In [265]:
# Start from the storage location, go to the lowest cost shop, then go to the lowest cost shop from there, and so on until all shops are visited

# Create a list of visited shops
visited = [0] # Start from the storage location

# Create a list of unvisited shops
unvisited = list(range(1, n))

# Calculate the cost of the route
cost = 0

# Van capacity
van_capacity = 300
remaining_capacity = van_capacity

In [266]:
active_arcs = []  # List to store paths for all trucks
current_arcs = []  # Current path for the active truck

while len(unvisited) > 0:
    current = 0
    while len(unvisited) >= 0:

        # Get the costs of the unvisited shops
        costs = cost_matrix.iloc[current, unvisited]

        # Filter out shops that exceed the remaining capacity
        feasible_shops = [shop for shop in unvisited if shop_demands[shop] <= remaining_capacity]

        if not feasible_shops:
            # If no feasible shops, end the current truck's route and reset
            next_shop = 0  # Go back to the storage location
            current_arcs.append((current, next_shop))
            active_arcs.append(current_arcs)
            current_arcs = []  # Reset the path for the next truck
            remaining_capacity = van_capacity  # Reset the capacity for the new truck
            break  # Exit the loop to start with a new truck

        # Get the costs of the feasible shops
        feasible_costs = costs[costs.index.astype(int).isin(feasible_shops)]

        # Get the index of the lowest cost shop
        next_shop = int(feasible_costs.idxmin())

        # Store the arc between current and next_shop
        current_arcs.append((current, next_shop))

        # Add the lowest cost shop to the visited list
        visited.append(next_shop)

        # Remove the lowest cost shop from the unvisited list
        unvisited.remove(next_shop)

        # Add the cost of the route
        cost += feasible_costs.min()

        # Update the remaining capacity
        remaining_capacity -= shop_demands[next_shop]

        # Move to the next shop
        current = next_shop


# Cost and route

In [267]:
# Calculate the cost of the route
print('Cost of the route:', cost)
print('Route:', active_arcs)

Cost of the route: 8
Route: [[(0, 1), (1, 4), (4, 2), (2, 0)], [(0, 3), (3, 0)]]
